In [91]:
# Import usual libraries
import numpy as np
import matplotlib.pyplot as plt
import itertools


import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import numpy as np

# 1 - DATA MANIPULATION
import pandas as pd
import numpy as np

# 2 - DATA VISUALISATION
import matplotlib.pyplot as plt
import seaborn as sns

# 3 - STATISTICS
from statsmodels.graphics.gofplots import qqplot

# 4 - MACHINE LEARNING
## 4.1 - Preprocessing

### 4.1.1 - Scalers
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

### 4.1.2 - Encoders
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

### 4.1.3 - Crossvalidation, Training, Model
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor

### 4.1.4 - Evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics  import ConfusionMatrixDisplay

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from math import sqrt

# Make all figures tiny for readability purpose
from matplotlib import rcParams
rcParams['figure.figsize'] = (5,3)
import macrobond_data_api as mda
from macrobond_data_api.web import WebClient

from macrobond_data_api.web import WebClient
from macrobond_data_api.common.types import SearchFilter

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd


In [92]:
Target = 'CPIAUCSL'
Drop = ['value_x','CPIAUCSL','core_cpi']

In [111]:
total_df = pd.read_csv('fbd_df_w_fred_gdp.csv', index_col='Unnamed: 0', parse_dates=True)
X = total_df.drop(columns=Drop)[:-21] # (8885, 651)  1990-01-01 to 2024-01-31 8871 non-null 
y = total_df[Target].values[:-21]
new_data = total_df.drop(columns=Drop).iloc[-1:]
y= y[-X.shape[0]:]

In [94]:
# # # Calculate the mean excluding NaN values
mean_val = np.nanmean(y)
# Replace NaN values with the mean
y[np.isnan(y)] = mean_val
y = pd.Series(y)



In [95]:
from sklearn.model_selection import train_test_split

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


In [96]:
## Non Linear PCA
# from sklearn.decomposition import KernelPCA
# from sklearn.pipeline import make_pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler

# # Updated preprocessing pipeline with Kernel PCA
# preproc = make_pipeline(
#     SimpleImputer(strategy='mean'),
#     StandardScaler(),
#     KernelPCA(n_components=None, kernel='rbf')  # Example with RBF kernel do not enter the % 
# ) 

In [97]:
# Linear PCA 
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from sklearn.decomposition import PCA

# Adjusted preprocessing pipeline with named steps
preproc = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95, random_state=42))  # Named step 'pca'
])


# # Feature selection remains the same
# # preproc_selector = Pipeline([
# #     ('preprocessing', preproc),  # Include the preprocessing steps with PCA
# #     ('feature_selection', SelectPercentile(
# #         mutual_info_regression,
# #         percentile=90 # Keep 90% of all features
# #     ))
# # ])

In [98]:

# model = XGBRegressor(random_state=42)

# param_distributions = {
#     'model__n_estimators': [200],  
#     'model__learning_rate': [0.1],  
#     'model__max_depth': [16, 20],  
# }

# pipe = Pipeline([
#     ('preprocessor', preproc),
#     ('model', model)
# ])

# random_search = RandomizedSearchCV(
#     pipe,
#     param_distributions=param_distributions,
#     n_iter=100,  # Number of parameter settings that are sampled. n_iter trades off runtime vs quality of the solution.
#     scoring='neg_mean_absolute_error',  # Assuming MSE is the metric of interest; adjust as needed.
#     cv=5,
#     verbose=2,
#     random_state=42,
#     n_jobs=-1  # Use all available cores
# )

# random_search.fit(X_train, y_train)

# print("Best parameters found: ", random_search.best_params_)
# print("Best score found: ", random_search.best_score_)


In [99]:
# genrally higher N_estimators and lower learning rates are more robust 
# Best parameters found:  {'model__n_estimators': 150, 'model__max_depth': 6, 'model__learning_rate': 0.1}
# Best score found:  -0.10133019738065827

# Defining the best model
model_best =  XGBRegressor(n_estimators=200, max_depth = 20, learning_rate = 0.1, random_state=42)

# Creating the pipeline with memory caching
pipe_best = make_pipeline(preproc, model_best)

# Assuming X_train and y_train are already defined and contain your training data
score = cross_val_score(pipe_best, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')

# Printing the standard deviation and mean of the cross-validation scores
print(score.std())
print(score.mean())

0.1076653324736503
-1.524087226392913


In [100]:
pipe_best.fit(X_train,y_train)
y_pred = pd.Series(pipe_best.predict(X_test))
y_test = pd.Series(y_test)

In [101]:

from statsmodels.tsa.stattools import acf

def forecast_accuracy(y_pred: pd.Series, y_true: pd.Series) -> float:

    mape = np.mean(np.abs(y_pred - y_true)/np.abs(y_true))  # Mean Absolute Percentage Error
    me = np.mean(y_pred - y_true)             # ME
    mae = np.mean(np.abs(y_pred - y_true))    # MAE
    mpe = np.mean((y_pred - y_true)/y_true)   # MPE
    rmse = np.mean((y_pred - y_true)**2)**.5  # RMSE
    corr = np.corrcoef(y_pred, y_true)[0,1]   # Correlation between the Actual and the Forecast
    mins = np.amin(np.hstack([y_pred.values.reshape(-1,1), y_true.values.reshape(-1,1)]), axis=1)
    maxs = np.amax(np.hstack([y_pred.values.reshape(-1,1), y_true.values.reshape(-1,1)]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(y_pred-y_true, fft=False)[1]                      # Lag 1 Autocorrelation of Error

    forecast = ({
        'mape':mape,
        'me':me,
        'mae': mae,
        'mpe': mpe,
        'rmse':rmse,
        'acf1':acf1,
        'corr':corr,
        'minmax':minmax
    })

    return forecast


In [102]:
forecast_accuracy(y_pred,y_test)

{'mape': 0.32115330375230644,
 'me': 45.047304088586486,
 'mae': 46.44604239935349,
 'mpe': 0.31193208262630406,
 'rmse': 53.81115218354335,
 'acf1': nan,
 'corr': 0.9992562418763288,
 'minmax': 0.005871602262185416}

In [112]:
# Make a prediction with the new observation
prediction = pipe_best.predict(new_data)[0]

print("Predicted value:", prediction)

Predicted value: 285.64145


In [104]:
model_forecast = pd.DataFrame()

# Assign rounded values of y_pred and y_test to the DataFrame
model_forecast['y_pred'] = y_pred.values
model_forecast['y_test'] = y_test.values

# Display the DataFrame
model_forecast

,y_pred,y_test
0,198.456528,199.700000
1,162.364014,162.800000
2,152.673340,150.512903
3,168.298355,164.700000
4,254.435532,254.277000
...,...,...
73,207.234177,207.454049
74,182.271179,181.200000
75,252.233490,251.663000
76,165.953003,165.900000


Viewing the PCA

In [105]:
# Assuming X_train and y_train are your features and target variables respectively
preproc.fit(X_train, y_train)


Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('pca', PCA(n_components=0.95, random_state=42))])

In [106]:
# Fit the preprocessing pipeline and transform X_train
X_train_transformed = preproc.fit_transform(X_train)

In [107]:
import pandas as pd

# Convert the transformed array into a DataFrame
transformed_df = pd.DataFrame(X_train_transformed,
                              columns=[f'Feature_{i}' for i in range(X_train_transformed.shape[1])])

# Display the transformed DataFrame
print(transformed_df)


     Feature_0  Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  \
0   -15.580499   2.693344   0.288657  -7.202345   1.155502  -0.581511   
1    -4.324043  -0.874573  -2.025151  -0.286253   7.275875  -3.706951   
2   -10.585483   9.410000   4.286161   5.350992  -1.949721   2.709703   
3    25.754696   6.026720   4.139323   0.027781  10.913109   6.300861   
4   -10.300227   9.932291   3.530113   6.089375  -1.856193   1.085880   
..         ...        ...        ...        ...        ...        ...   
305  12.846466  -8.692559  -1.520787  -1.160055   4.637819   5.965863   
306 -18.903531   5.409381   1.009240  -9.047834  -1.598162  -2.112807   
307   7.496317 -14.345002  -5.633839  -4.835230  -5.845390  -0.117176   
308 -18.090808   7.018423   2.736675  -8.158891  -7.647230  -1.626803   
309 -17.005506   3.915885   1.973801  -8.465825  -1.310751   0.374418   

     Feature_6  Feature_7  Feature_8  Feature_9  ...  Feature_19  Feature_20  \
0    -1.206552   1.038941  -2.142024   0.13

In [108]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import shap
from xgboost import XGBRegressor

# Fit your model on the transformed data
model = XGBRegressor(n_estimators=200, max_depth=20, learning_rate=0.1, random_state=42)
model.fit(X_train_transformed, y_train)

# Compute SHAP values to identify influential components
explainer = shap.Explainer(model)
shap_values = explainer(X_train_transformed)

# Assuming we're interested in the top 1 component for demonstration
top_components = np.argsort(-np.sum(np.abs(shap_values.values), axis=0))[0:1]

# Extract PCA step from the pipeline
pca = preproc.named_steps['pca']

# Extract PCA loadings for the top influential components
loadings = pca.components_[top_components]

# Assuming X_train is a DataFrame and has column names
feature_names = X_train.columns

# Create a DataFrame for loadings
loading_df = pd.DataFrame(loadings, columns=feature_names, index=[f'Component {i+1}' for i in top_components])

print("PCA Loadings for the top influential components:")
print(loading_df)

PCA Loadings for the top influential components:
                value  population   value_y  value_x.2  value_CES0500000003  \
Component 1  0.060905    0.059212  0.053602   0.061492             0.037739   

             value_y.2  value_GDPNOW  value_A191RL1Q225SBEA  \
Component 1   0.000544      0.002172              -0.000266   

             Monthly Nominal GDP Index  Monthly Real GDP Index  ...  \
Component 1                   0.062057                0.059964  ...   

             usinea1986  uslama4503  usinea1980  usinea1968  usinea1122  \
Component 1    0.004203    0.035704   -0.006708   -0.006523    0.009303   

             usinea1104  usinea1113  usinea1089  usinea1092       NFP  
Component 1    0.003292    0.000627    0.002756    0.003177  0.053602  

[1 rows x 676 columns]


In [110]:
# Assuming loading_df is already created and contains the PCA loadings for Component 1
# Extract loadings for Component 1
component_1_loadings = loading_df.loc['Component 1']

# Sort the loadings by their absolute values in descending order to find the top features
sorted_loadings = component_1_loadings.abs().sort_values(ascending=False)

# Select the top 20 features based on their loadings
top_50_features = sorted_loadings.head(50)

# Convert the series to a list of feature names
top_50_feature_names = top_50_features.index.tolist()

# Optionally, to get the corresponding loading values as well
top_50_feature_values = top_50_features.values.tolist()

print("Top 50 PCA Loadings for Component 1 (Features):", top_50_feature_names)
print("Top 50 PCA Loadings for Component 1 (Values):", top_50_feature_values)

Top 50 PCA Loadings for Component 1 (Features): ['uslama02532', 'uslama02531', 'uslama02530', 'value_M2REAL', 'bea020_76w729rc_m', 'bea022_76w826rc_m', 'bea018_76w823rc_m', 'usinea2003', 'usinea1997', 'usinea1042', 'bea012_76a048rc_m', 'usinea1009', 'uslama02529', 'bea001_76a065rc_m', 'bea019_76w824rc_m', 'usinea1039', 'usinea1006', 'bea027_76a067rc_m', 'uslama02562', 'usinea1037', 'uslama0254', 'uslama0255', 'usinea1000', 'usinea1033', 'usinea1002', 'usinea1035', 'uslama0244', 'uslama0245', 'uslama0252', 'uslama0253', 'usinea1993', 'usinea1999', 'uslama0226', 'uslama0227', 'usinea1004', 'bea038_76a229rc_m', 'usinea1998', 'oecd_mei_00438733', 'oecd_mei_00438732', 'uslama0218', 'uslama0219', 'uslama0222', 'uslama0223', 'usinea2000', 'uslama4890', 'usinea2002', 'uslama00916', 'usinea1994', 'usinea1992', 'uslama00915']
Top 50 PCA Loadings for Component 1 (Values): [0.06335713492990883, 0.06334114177535607, 0.06333857581834945, 0.06315393516128621, 0.06310263186209304, 0.06308512662753733,